In [14]:
import numpy as np
import pandas as pd
import pandas_datareader as web
from datetime import datetime
from pandas_datareader._utils import RemoteDataError
import requests
import json
import yfinance as yf

In [15]:
data=pd.read_csv("C:\\Users\\Kalpa\\Documents\\PythonFiles\\Stocklist.csv")
#data=pd.read_csv("C:\\Users\\Kalpa\\Documents\\PythonFiles\\finalcountdown.csv")
#data=pd.read_csv("C:\\Users\\Kalpa\\Documents\\PythonFiles\\Megacap.csv")
data.head()
#end = datetime.today()
#start = datetime(end.year -5, end.month, end.day)
#print(start)

,Symbol
0,NESTLEIND.NS
1,ICICIBANK.NS
2,ASIANPAINT.NS
3,DMART.NS
4,BATAINDIA.NS


In [16]:
# Pull stock data from Yahoo finance 

for i in range (len(data)):
 
    stock = web.DataReader(data.loc[i,"Symbol"],data_source='yahoo',start='13/4/2015', end='28/4/2020')
    detail=yf.Ticker(data.loc[i,"Symbol"])
    stock['Log_Ret'] = np.log(stock['Close'] / stock['Close'].shift(1))
    stock['Volatility1yr']=stock['Log_Ret'].rolling(252).std()
    stock['Volatility3yr']=stock['Log_Ret'].rolling(756).std()
    stock['1yrAvgRet']=stock['Log_Ret'].rolling(252).mean()
    stock['3yrAvgRet']=stock['Log_Ret'].rolling(756).mean()
    #Dividend Data
    df=detail.get_dividends().to_frame()
    #populate data
    data.loc[i,"1yrAvgRet"]=stock['1yrAvgRet'][-1]
    data.loc[i,"3yrAvgRet"]=stock['3yrAvgRet'][-1]
    data.loc[i,"1yrDivRet"]=(1/252)*(df[df.index>"2019"].sum()/stock['Close'][-252 if len(stock['Close'])>252 else -len(stock['Close'])])[0]
    data.loc[i,"3yrDivRet"]=(1/756)*(df[df.index>"2017"].sum()/stock['Close'][-756 if len(stock['Close'])>756 else -len(stock['Close'])])[0]
    data.loc[i,"1yrSharpe"]=((stock['1yrAvgRet'][-1]+data.loc[i,"1yrDivRet"])/stock['Volatility1yr'][-1])*np.sqrt(252)
    data.loc[i,"3yrSharpe"]=((stock['3yrAvgRet'][-1]+data.loc[i,"3yrDivRet"])/stock['Volatility3yr'][-1])*np.sqrt(252)
    #format in percentage and annualize
    data.loc[i,"1yrAvgRet"]="{:.2f}%".format(data.loc[i,"1yrAvgRet"]*100*252)
    data.loc[i,"3yrAvgRet"]="{:.2f}%".format(data.loc[i,"3yrAvgRet"]*100*252)
    data.loc[i,"1yrDivRet"]="{:.2f}%".format(data.loc[i,"1yrDivRet"]*100*252)
    data.loc[i,"3yrDivRet"]="{:.2f}%".format(data.loc[i,"3yrDivRet"]*100*252)

In [17]:
    # Compute Volatility using the pandas rolling standard deviation function
#stock['Volatility']=stock['Log_Ret'].rolling(252).std()
#data.to_csv("C:\\Users\\Kalpa\\Documents\\PythonFiles\\book.csv")
k1=pd.merge(data.nlargest(30,'1yrSharpe'),data.nlargest(30,'3yrSharpe')['Symbol'],on='Symbol')
k2=data[data['1yrAvgRet']>data['3yrAvgRet']]
pd.merge(k1,k2['Symbol'],on='Symbol')

,Symbol,1yrAvgRet,3yrAvgRet,1yrDivRet,3yrDivRet,1yrSharpe,3yrSharpe
0,NESTLEIND.NS,49.00%,33.14%,1.38%,1.91%,1.570001,1.352641
1,DMART.NS,44.84%,43.31%,0.00%,0.00%,1.174167,1.221533
2,ASIANPAINT.NS,24.84%,18.17%,1.26%,1.10%,0.806741,0.761505
3,RELIANCE.NS,5.86%,27.66%,0.48%,1.25%,0.143769,0.884925


In [18]:
# Plot the stock Price series and the Volatility
#stock[['Close', 'Volatility']].plot(subplots=True, color='blue',figsize=(8, 6))

In [19]:
#"{:.2%}".format(oneyrreturn)
#(oneyrreturn)/stock['Volatility'][-1]
highlow=np.min(stock[stock.index>"2019"]["Close"])/np.min(stock[(stock.index>"2017") & (stock.index<"2018")]["Close"])-1


In [20]:
i

6

In [21]:
data[data['1yrAvgRet']>data['3yrAvgRet']]

,Symbol,1yrAvgRet,3yrAvgRet,1yrDivRet,3yrDivRet,1yrSharpe,3yrSharpe
0,NESTLEIND.NS,49.00%,33.14%,1.38%,1.91%,1.570001,1.352641
2,ASIANPAINT.NS,24.84%,18.17%,1.26%,1.10%,0.806741,0.761505
3,DMART.NS,44.84%,43.31%,0.00%,0.00%,1.174167,1.221533
6,RELIANCE.NS,5.86%,27.66%,0.48%,1.25%,0.143769,0.884925
